In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# loan level data
loanlevel_file = r"/Users/xiaoyu/Downloads/fsq-21-1gr-boe-aug-21.csv"

In [3]:
import pandas as pd
import toolz as tz
from lenses import lens

In [4]:
loanAssetDF = pd.read_csv(loanlevel_file)

/var/folders/pb/0lsr_g9s0x340sd74jhjdhj40000gn/T/ipykernel_41295/531353068.py:1: DtypeWarning: Columns (20,27,72,106,107,117,119,136) have mixed types. Specify dtype option on import or set low_memory=False.
  loanAssetDF = pd.read_csv(loanlevel_file)


In [5]:
from dateparser import *
from dateutil import relativedelta

In [6]:
def converQtoDate(y):
    d = y.replace("Q1","03-30").replace("Q2","06-30").replace("Q3","09-30").replace("Q4","12-31")
    return parse(d)

In [7]:
def mapping(x:dict):
    freqMapping = {
        1:"Monthly"
    }
    repayMapping = {
        1:"Level"
        ,2:"Level"
        ,6:"I_P"
    }
    statusMapping = {
        1:"Current",
        2:"Defaulted",
        4:"Defaulted"
    }
    
    def converQtoDate(y):
        d = y.replace("Q1","03-30").replace("Q2","06-30").replace("Q3","09-30").replace("Q4","12-31")
        return parse(d)
    
    def calDateAndTerm(s,oTerm,cutoffDate):
        sd = converQtoDate(s)
        cutoffDate = parse(cutoffDate)
        age = relativedelta.relativedelta(cutoffDate,sd)
        monthsPast = age.years*12+age.months
        
        return (sd.strftime("%Y-%m-%d")
                , oTerm - monthsPast)
    
    originDate,remainTerm = calDateAndTerm(x["AR55"],x['AR61'],x['AR1'])
    
    
    return ["Mortgage"
              ,{"originBalance": x['AR66']
                ,"originRate": ["fix",x['AR109']]
                ,"originTerm": x['AR61']
                ,"freq": freqMapping[x['AR70']]
                ,"type": repayMapping[x['AR72']]
                ,"originDate": originDate}
              ,{"currentBalance": x['AR67']
                ,"currentRate": x['AR109']
                ,"remainTerm": remainTerm
                ,"status": statusMapping[x['AR166']]
               }
           ]

In [24]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.41.3

In [25]:
def mergeAssets(xs):
    cbals = lens.Each()[2]['currentBalance'].collect()(xs)
    crates = lens.Each()[2]['currentRate'].collect()(xs)
    obals = lens.Each()[1]['originBalance'].collect()(xs)
    orates = lens.Each()[1]['originRate'][1].collect()(xs)

    ot = xs[0][1]['originTerm']
    rt = xs[0][2]['remainTerm']
    sd = xs[0][1]['originDate']
    st = xs[0][2]['status']
    freq = xs[0][1]['freq']
    t = xs[0][1]['type']
    totalCurBal = sum(cbals)
    totalOriBal = sum(obals)
    
    if totalCurBal == 0:
        return None
    avgRate = sum([ x*y for x,y in zip(cbals,crates)])/totalCurBal
    avgOriRate = sum([ x*y for x,y in zip(obals,orates)])/totalOriBal

    return ['Mortgage',
               {'originBalance': totalOriBal,
                'originRate': ['fix', 0.01],
                'originTerm': ot,
                'freq': freq,
                'type': t,
                'originDate': sd},
               {'currentBalance': totalCurBal,
                'currentRate': avgRate,
                'remainTerm': rt,
                'status': st}]

def consolAsset(ast_list):
    return tz.pipe(tz.groupby(lambda x:(x[1]['originTerm'],x[1]['freq'],x[1]['type'],x[1]['originDate'],x[2]['remainTerm'],x[2]['status']), ast_list)
                   ,lambda m: tz.valmap(lambda x: mergeAssets(x), m)
                   ,lambda m: [ _ for _ in m.values() if _ and _[2]['status']!="Defaulted"]
                   )

In [87]:
loanAssetInMap = loanAssetDF.to_dict('records')
myPool = {"assets": [ mapping(_) for _ in loanAssetInMap  ], "cutoffDate":"2021-08-31" }

In [93]:
myPool['assets'][:10]

[['Mortgage',
  {'originBalance': 106250.0,
   'originRate': ['fix', 0.0499],
   'originTerm': 420,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2021-03-30'},
  {'currentBalance': 105557.37,
   'currentRate': 0.0499,
   'remainTerm': 415,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 79499.0,
   'originRate': ['fix', 0.0414],
   'originTerm': 360,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2021-03-30'},
  {'currentBalance': 78537.94,
   'currentRate': 0.0414,
   'remainTerm': 355,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 104999.0,
   'originRate': ['fix', 0.0384],
   'originTerm': 276,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '2021-03-30'},
  {'currentBalance': 103806.21,
   'currentRate': 0.0384,
   'remainTerm': 271,
   'status': 'Current'}],
 ['Mortgage',
  {'originBalance': 69999.0,
   'originRate': ['fix', 0.0384],
   'originTerm': 180,
   'freq': 'Monthly',
   'type': 'Level',
   'originDate': '202

In [89]:
issueDate = "2021-06-28"
stepUpDate = "2026-06-01"
firstPayDate = "2021-09-16"
bonds = [
     ("A",{"balance":639_375_000
             ,"rate":0.07
             ,"originBalance":639_375_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.0065,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.00325)
             ,"bondType":{"PAC":
                          [[ "2021-09-16",626651437.5]
                           ,[ "2021-12-16",617252625]
                           ,[ "2022-03-16",607981687.5]
                           ,[ "2022-06-16",598838625]
                           ,[ "2022-09-16",589823437.5]
                           ,[ "2022-12-16",582342750]
                           ,[ "2023-03-16",577995000]
                           ,[ "2023-06-16",573647250]
                           ,[ "2023-09-16",569363437.5]
                           ,[ "2023-12-16",565015687.5]
                           ,[ "2024-03-16",560795812.5]
                           ,[ "2024-06-16",556575937.5]
                           ,[ "2024-09-16",552356062.5]
                           ,[ "2024-12-16",548136187.5]
                           ,[ "2025-03-16",543980250]
                           ,[ "2025-06-16",539888250]
                           ,[ "2025-09-16",535732312.5]
                           ,[ "2025-12-16",531704250]
                           ,[ "2026-03-16",527612250]
                           ,[ "2026-06-16",0]]
                         }}),
     ("B",{"balance":65_625_000
             ,"rate":0.07
             ,"originBalance":65_625_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.01,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.005)
             ,"bondType":{"Sequential":None}}),
     ("C",{"balance":22_500_000
             ,"rate":0.07
             ,"originBalance":22_500_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.0125,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.00525)
             ,"bondType":{"Sequential":None}}),
     ("D",{"balance":22_500_000
             ,"rate":0.07
             ,"originBalance":22_500_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.03,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.00325)
             ,"bondType":{"Sequential":None}}),
     ("X1",{"balance":33_750_000
             ,"rate":0.07
             ,"originBalance": 33_750_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.0525,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.00325)
             ,"bondType":{"Sequential":None}}),
     ("X2",{"balance":18_750_000
             ,"rate":0.07
             ,"originBalance":18_750_000
             ,"originRate":0.07
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.0525,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.00325)
             ,"bondType":{"Sequential":None}}),
    ("Z",{"balance":15_000_000
             ,"rate":0.0
             ,"originBalance":15_000_000
             ,"originRate":0.00
             ,"startDate":issueDate
             ,"rateType":{"floater":[0.05, "SOFR1Y",0.05,"MonthEnd"]}
             ,"stepUp":("once",stepUpDate,0.025)
             ,"bondType":{"Sequential":None}
             }),
    ("Certificateholders",{"balance":1.00
              ,"rate":0.0
              ,"originBalance":1.00
              ,"originRate":0.00
              ,"startDate":issueDate
              ,"rateType":{"fix":0.0}
              ,"bondType":{"Equity":None} }
    )
        ]

In [56]:
accounts = [
    ("general_reserve",{"balance":15_000_000,
                            "type":{"when":[
                                    [("isPaidOff", "A", "B", "C"),False],
                                    {"targetReserve":[("bondBalance","A","B","C","D"),0.02]},
                                    {"fixReserve":0},
                                    ]
                                }
                       })
    ,("revenue_account",{"balance":0})
    ,("principal_account",{"balance":0})
    ,("retain_prin_ledger",{"balance":0})
]

In [57]:
fees = (
    ("baseFee",{"type":{"annualPctFee":[("poolBalance",),0.0015]}})
    ,("issuerProfit",{"type":{"byPeriod":1500}})
    ,("otherFee", {"type":{"recurFee":["QuarterEnd",75_000/4]}})
    ,("admission",{"type":{"fixFee":11740}})
)

In [58]:
waterfall = {
    "default":[
        ["transfer","general_reserve","revenue_account",{"reserve":"excess"}],
        ["calcAndPayFee", "revenue_account", ["otherFee","admission"]],
        ["calcAndPayFee", "revenue_account", ["baseFee"]], 
        # Fees and expenses of the Mortgage Administrator
        ["calcAndPayFee", "revenue_account", ["issuerProfit"]],# issuer proft
        #["settleSwap","revenue_account","swap-a"],# amt to IRS (ex Swap Suborinated Amount and Swap Excluded Payable Amounts)
        ["accrueAndPayInt", "revenue_account", ["A"] ],# Interst on A-Green
        ["payPrin","revenue_account",["A"],{"limit":{"formula": ("ledgerBalance","pdl-A")}}],# A PDL
        ["accrueAndPayInt", "revenue_account", ["B"] ],# Int B
        ["payPrin","revenue_account",["B"],{"limit":{"formula": ("ledgerBalance","pdl-B")}}],# B PDL
        ["accrueAndPayInt", "revenue_account", ["C"] ],# Int C
        ["payPrin","revenue_account",["C"],{"limit":{"formula": ("ledgerBalance","pdl-C")}}],# C PDL
        ["transfer", "revenue_account", "general_reserve", {"reserve":"gap"}],# General Reserve Fund Ledger up to Require Amount 
        ["payPrin","revenue_account",["D"],{"limit":{"formula": ("ledgerBalance","pdl-D")}}],# D PDL
        ["accrueAndPayInt", "revenue_account", ["D"] ],# Int D
        ["accrueAndPayInt", "revenue_account", ["X1"] ],# Int X1
        ["payPrin","revenue_account",["X1"]],# Prin X1
        ["If",[("isPaidOff","X1"),True],
          ["accrueAndPayInt", "revenue_account", ["X2"] ]],# Int X2 (if X1 is off)
        ["payPrin","revenue_account",["X2"]],# Prin X2
        ["payFee", "revenue_account", ["baseFee"]],# service Fee above cap
        ["accrueAndPayInt", "revenue_account", ["Z"] ],# Int Z
        # Pirn to Z if A-C is off and X1 X2 is outs
        ["If",[("isPaidOff","A","B","C","X1","X2"),True]
          ,["payPrin", "revenue_account", ["Z"] ]],        
        ["payIntResidual","revenue_account","Certificateholders"],# surplus to Certificated holders
        ["payPrin","principal_account",["A"]],# in futher sale period, pay A till target notional
        ["IfElse",["date","in",firstPayDate]
                 ,[["payPrin","principal_account",["A","B","C","D"]]]
                 ,[["payPrinBySeq","principal_account",["A","B","C","X1","X2"]]]
        ],
        ["payIntResidual","principal_account","Certificateholders"],# Surplus to certificate holder
    ],
    "endOfCollection":[["bookBy",["PDL"
                                  ,"Debit"
                                  ,("curPoolCollection", None, "Defaults")
                                  ,[("pdl-D",("bondBalance","D"))
                                    ,("pdl-C",("bondBalance","C"))
                                    ,("pdl-B",("bondBalance","B"))
                                    ,("pdl-A",("bondBalance","A"))]]]
                     ]
}

In [59]:
pdl_ledger = {
    "pdl-A":{"balance":0},"pdl-B":{"balance":0},"pdl-C":{"balance":0},"pdl-D":{"balance":0}
}

swap = {
    "swap-a":{}
}

In [60]:
dates = {"firstPay":firstPayDate
         ,"closing":"2021-06-28"
         ,"payFreq":["EveryNMonth","2021-12-16",3]
         ,"poolFreq":"MonthEnd"
         ,"cutoff":"2021-08-31"
         ,"stated":"2070-01-01"}

collectionRule = [["CollectedInterest","revenue_account"]
                  ,["CollectedPrincipal","principal_account"]
                  ,["CollectedPrepayment","principal_account"]
                  ,["CollectedRecoveries","principal_account"]]

In [90]:
from absbox import Generic,mkDeal

d = Generic(
    "FSQ"
    ,dates
    ,myPool
    ,accounts
    ,bonds
    ,fees
    ,waterfall
    ,collectionRule
    ,None
    ,None
    ,None
    ,None
    ,"Amortizing"
    ,None # optional
    ,pdl_ledger # optional
    ,None # optional
)

In [94]:
d.json

{'tag': 'MDeal',
 'contents': {'dates': {'tag': 'PreClosingDates',
   'contents': ['2021-08-31',
    '2021-06-28',
    None,
    '2070-01-01',
    ['2021-06-28', {'tag': 'MonthEnd'}],
    ['2021-09-16', {'tag': 'EveryNMonth', 'contents': ['2021-12-16', 3]}]]},
  'name': 'FSQ',
  'status': {'tag': 'Amortizing'},
  'pool': {'tag': 'MultiPool',
   'contents': {'PoolConsol': {'assets': [{'tag': 'Mortgage',
       'contents': [{'originBalance': 106250.0,
         'originRate': {'tag': 'Fix', 'contents': ['DC_ACT_365F', 0.0499]},
         'originTerm': 420,
         'period': 'Monthly',
         'startDate': '2021-03-30',
         'prinType': {'tag': 'Level'},
         'prepaymentPenalty': None,
         'obligor': None,
         'tag': 'MortgageOriginalInfo'},
        105557.37,
        0.0499,
        415,
        None,
        {'tag': 'Current'}]},
      {'tag': 'Mortgage',
       'contents': [{'originBalance': 79499.0,
         'originRate': {'tag': 'Fix', 'contents': ['DC_ACT_365F', 0.0

In [92]:
%%timeit 

r = localAPI.run(d,
                 runAssump=[("interest",("SOFR1Y",0.06))],read=True)

EngineError: Unexpected "NaN]}, \"originTerm\": 300, \"per", expecting JSON value

In [95]:
%%timeit -n 3

rp = localAPI.runPool(myPool,read=True)

EngineError: Unexpected "NaN]}, \"originTerm\": 300, \"per", expecting JSON value

In [85]:
%%timeit

rp = localAPI.runPool(myPool,read=True)

534 ms ± 5.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
8.23 / 0.534

15.411985018726591

In [80]:
myPool & lens['assets'].modify(lambda xs: xs[:100])

UnboundLens(TrivialIso())

In [81]:
%%timeit -n 3

rp = localAPI.runPool(myPool & lens['assets'].modify(lambda xs: xs[:100]),read=True)

3.94 s ± 5.47 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [65]:
r['fees']['baseFee']

,balance,payment,due
date,,,
2021-09-16,141027.10,0.00,0
2021-12-16,0.00,305987.42,0
2022-03-16,0.00,162018.20,0
2022-06-16,0.00,165607.82,0
2022-09-16,0.00,164816.44,0
...,...,...,...
2060-09-16,0.00,1603.05,0
2060-12-16,0.00,1550.63,0
2061-03-16,0.00,1264.78,0
